In [115]:
%%typecheck

from pprint import pprint
from math import fsum,sqrt
from typing import Iterable, Tuple, List,Sequence,Dict,DefaultDict
from dis import dis
from collections import defaultdict
from functools import partial
from random import sample

Point = Tuple[int,...]
Centroid = Point


points: List[Point] = [
        (10, 41, 23),
        (22, 30, 29),
        (11, 42, 5),
        (20, 32, 4),
        (12, 40, 12),
        (21, 36, 23),
    ]

def transpose(data: Sequence[Tuple[int,...]]):
    return list(zip(*data))

def mean(data: Iterable[float])-> float:
    data = list(data)
    return fsum(data)/len(data)
    
def dist(p:Point,q:Point, fsum=fsum, sqrt=sqrt, zip=zip)-> float:
    return sqrt(fsum( (p-q)**2 for p,q in zip(p,q))) 

def assign_data(centroids: Sequence[Centroid] , data: Iterable[Point])-> Dict[Centroid, List[Point]]:
    d:DefaultDict = defaultdict(list)
    
    for point in data:        
        closest_centroid = min(centroids, key= partial( dist, point))
        d[closest_centroid].append(point)
    return dict(d)

def compute_centroids(groups:Iterable[Sequence[Point]])->List[Centroid]:
    return[tuple(map(mean,transpose(group))) for group in groups]

def k_means(data: Iterable[Point], k:int=3, iterations:int=1000)-> List[Centroid]:
    data = list(data)
    centroids = sample(data, k)
    for i in range(iterations):        
        labeled = assign_data(centroids, data)
        centroids = compute_centroids(labeled.values())
    return centroids

    
    

# dis(dist.__code__)

groups= [
    [(10, 41, 23),
     (11, 42, 5),
     (20, 32, 4),
     (12, 40, 12)],
    [(22, 30, 29), 
     (21, 36, 23)]
]

centroids = k_means(points)
d = assign_data(centroids, points)
pprint(d)

<string>:38: error: No overload variant of "min" matches argument types [typing.Sequence[builtins.tuple[builtins.int]], functools.partial[builtins.float*]]
<string>:43: error: Argument 1 to "map" has incompatible type "Callable[[Iterable[float]], float]"; expected "Callable[[Any], int]"

{(11.5, 41.0, 8.5): [(11, 42, 5), (12, 40, 12)],
 (17.666666666666668, 35.666666666666664, 25.0): [(10, 41, 23),
                                                  (22, 30, 29),
                                                  (21, 36, 23)],
 (20.0, 32.0, 4.0): [(20, 32, 4)]}
